# **ASGSA Tools of the Trade** - MetPy

In this Tools of the Trade, we will be investigating a Python library called **MetPy** (https://unidata.github.io/MetPy/latest/index.html) and using it to plot skew-T log-p data.

The input includes a text file with the data you wish to plot.

The output includes a skew-T log-p diagram with:

- The plotted temperature, dewpoint, and wind profiles
- A hodograph
- Basic severe weather variables

***NOTE:*** *This code is written to work with data from the University of Wyoming Upper Air website. Adjustments will need to be made to work with data from other sources!*

For questions relating to this code, please contact Alec Sczepanski (alec.sczepanski@und.edu).

Without further ado, here we go!

***

To begin, we need data. A data file has been given to you with to go with this tutorial, but other data can be retrieved from the following website: https://weather.uwyo.edu/upperair/sounding.html. Note: This website only contains archived NWS radiosonde data.

Choose the date and time for the data you want from the respective drop-down menus. Make sure 'Type of Plot' is set to 'Text: List'. Click on the location you wish to get your data from.

A new tab should open with the data. Copy the data from the first dashed line to the very last line of data and paste into a Notepad editor or other text editor.

Save the file as a .txt into a folder you will remember.

***

Now that we have data, let's start with importing necessary modules:
- `pandas`: will be used to read in an parse the input data file
- `matplotlib`: will be used to create axes to plot the data onto
- `inset_axes`: an mpl_toolkits.axes_grid1.inset_locator sub-library, will be used to create axes for the hodograph
- `metpy`: will be used to process and plot data
- `mpcalc`: a metpy sub-library, will be used to manipulate data
- `SkewT, Hodograph`: metpy.plots sub-libraries, will be used to plot skew-T and hodograph
- `units`: a metpy.units sub-library, will be used to assign units to the data
- `numpy`: one can never forget to import numpy. Never.
- `math`: will be used to determine if there are missing data
- `warnings`: will be used to ignore any warnings raised

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import metpy
import metpy.calc as mpcalc
from metpy.plots import SkewT, Hodograph
from metpy.units import units
import numpy as np
import math as m

# Include ability to ignore warnings that do no break the code:
import warnings
warnings.filterwarnings("ignore")

Next, we will read in the data. For the sake of simplicity we will read in data one file at a time, however this code can easily be manipulated to create a function that goes through many files at once.

We will first establish new column names for the columns we will be pulling from the data file. This includes:
- Pressure
- Height
- Temperature
- Dewpoint
- Wind direction
- Wind speed

In [ ]:
col_names = ['pressure', 'height', 'temperature', 'dewpoint', 'direction', 'speed']

# Read in the data:
df = pd.read_csv('ABR_20210810_00.txt',
                 delim_whitespace = True, #set the deliminator to that any space/tab will indicate a separation between columns
                 skiprows = 5, #skip the first 5 rows; this includes the header and the arbitrary 1000 hPa row
                 usecols = [0,1,2,3,6,7], #use the given columnns; these match up to the names established in col_names
                 names = col_names) #rename the columns with those given in col_names

Once we have the data read in, we can assign units to each column using metpy.units (which we've imported as 'units'). 

While we're at it, we we also calculate the u and v components of the wind. This is necessary to plot wind barbs later on.

In [ ]:
p = df['pressure'].values * units.hPa
T = df['temperature'].values * units.degC
Td = df['dewpoint'].values * units.degC
wind_speed = df['speed'].values * units.knots #NOTE: for some data sets, wind speed is in m/s. If this is the case, do: df['speed'].values * 1.994 * units.knots
wind_dir = df['direction'].values * units.degrees

# Calculate u and v components of the wind:
u, v = mpcalc.wind_components(wind_speed, wind_dir)

By looking at the data, you'll notice that there are quite a number of data points in the vertical. For temperature and dewpoint, that's fantastic. But for winds, it can be painful to try and interpret that many points on a skew-T. 

So, we will re-sample the pressure data using mpcalc.resample_nn_1d (one-dimensional nearest-neighbor) to get new data points approximately every 50 hPa. We will use this interval later to pull winds from approximately every 50 hPa.

In [ ]:
# Calculate an interval in which to plot wind barbs (1050 to 100 hPa, every 50 hPa) and then resample pressure data:
interval = np.arange(100, 1050, 50) * units.hPa

# Resample pressure to the given interval:
p_index = mpcalc.resample_nn_1d(p, interval)

Now we have everything we need to plot a basic skew-T log-p!

Let's start plotting now by using `SkewT`, which we imported from metpy.plots.

In [ ]:
fig = plt.figure(figsize=(9,10))

skew = SkewT(fig, rotation = 45) #rotate the temp axis by 45 degrees

# Plot temperature data:
skew.plot(p, T, color = 'red', linewidth = 2)

# Plot dewpoint data:
skew.plot(p, Td, color = 'green', linewidth = 2)

# Plot wind data:
skew.plot_barbs(p[p_index], u[p_index], v[p_index], color = 'black')

# Set axis limits:
skew.ax.set_ylim(1025, 100)
skew.ax.set_xlim(-40, 60)

# Set axis labels:
plt.xlabel(r'Temperature ($\degree$C)')
plt.ylabel('Log Pressure (hPa)')

plt.show()

***
A very nice skew-T! But it feels like it's missing some things.

Let's add the adiabats and the mixing ratio lines:

In [ ]:
fig = plt.figure(figsize=(9,10))

skew = SkewT(fig, rotation = 45) #rotate the temp axis by 45 degrees

#######################################
# Plot adiabats, mixing ratio lines:
skew.plot_dry_adiabats(linewidth = 1)
skew.plot_moist_adiabats(linewidth = 1)
skew.plot_mixing_lines(linewidth = 1)
#######################################

# Plot temperature data:
skew.plot(p, T, color = 'red', linewidth = 2)

# Plot dewpoint data:
skew.plot(p, Td, color = 'green', linewidth = 2)

# Plot wind data:
skew.plot_barbs(p[p_index], u[p_index], v[p_index], color = 'black')

# Set axis limits:
skew.ax.set_ylim(1025, 100)
skew.ax.set_xlim(-40, 60)

# Set axis labels:
plt.xlabel(r'Temperature ($\degree$C)')
plt.ylabel('Log Pressure (hPa)')

plt.show()

***
Nifty! But it still feels like it's missing something. Perhaps a hodograph?

Let's create a hodograph on a separate axis and inset it onto the skew-T:

In [ ]:
fig = plt.figure(figsize=(9,10))

skew = SkewT(fig, rotation = 45) #rotate the temp axis by 45 degrees

# Plot adiabats, mixing ratio lines:
skew.plot_dry_adiabats(linewidth = 1)
skew.plot_moist_adiabats(linewidth = 1)
skew.plot_mixing_lines(linewidth = 1)

# Plot temperature data:
skew.plot(p, T, color = 'red', linewidth = 2)

# Plot dewpoint data:
skew.plot(p, Td, color = 'green', linewidth = 2)

# Plot wind data:
skew.plot_barbs(p[p_index], u[p_index], v[p_index], color = 'black')

# Set axis limits:
skew.ax.set_ylim(1025, 100)
skew.ax.set_xlim(-40, 60)

# Set axis labels:
plt.xlabel(r'Temperature ($\degree$C)')
plt.ylabel('Log Pressure (hPa)')

#################################################################
# Add a hodograph:
ax_hod = inset_axes(skew.ax, '30%', '30%', loc = 1) #create hodograph axes to go in upper-right corner of skew-T, 30% of skew-T size
h = Hodograph(ax_hod, component_range = 80) #make hodograph with range of 80 kts
h.add_grid(increment = 20) #add grid of 20-kt increments
h.plot_colormapped(u[p_index], v[p_index], wind_speed[p_index])
#################################################################

plt.show()

***
Wild! It looks really good and acts as a brilliant standalone skew-T, but we can add some things to it.

Next, we can calculate some indices including CAPE, MUCAPE, CIN, LCL, LFC, EL, and a parcel profile. We will then add these numbers to a table underneath the skew-T.

Lastly, we will draw the parcel profile and shade in areas of CAPE and CIN.

In [ ]:
# Calculate some indices:
parcel_profile = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')

# LCL
LCL_pressure, LCL_temperature = mpcalc.lcl(p[0], T[0], Td[0])
LCL_pressure = str(LCL_pressure)[:-11].strip() #strip off last 11 characters, 'hectoPascals'

if m.isnan(float(LCL_pressure)) == True: #check to see if LCL is NaN
    LCL_pressure = 'NaN'
    
else:
    LCL_pressure = int(float(LCL_pressure)) #return LCL pressure as an integer
    
# LFC
LFC_pressure, LFC_temperature = mpcalc.lfc(p, T, Td)
LFC_pressure = str(LFC_pressure)[:-11].strip()

if m.isnan(float(LFC_pressure)) == True:
    LFC_pressure = 'NaN'
    
else:
    LFC_pressure = int(float(LFC_pressure))
    
# EL
EL_pressure, EL_temperature = mpcalc.el(p, T, Td)
EL_pressure = str(EL_pressure)[:-11].strip()

if m.isnan(float(EL_pressure)) == True:
    EL_pressure = 'NaN'
    
else:
    EL_pressure = int(float(EL_pressure))
    
# CAPE, CIN, MUCAPE
CAPE, CIN = mpcalc.cape_cin(p, T, Td, parcel_profile = parcel_profile)
CAPE = str(CAPE)[:-16].strip()
CAPE = int(float(CAPE))
CIN = str(CIN)[:-16].strip()
CIN = int(float(CIN))

MUCAPE, MUCIN = mpcalc.most_unstable_cape_cin(p, T, Td)
MUCAPE = str(MUCAPE)[:-16].strip()
MUCAPE = int(float(MUCAPE))

In [ ]:
# PLOT THE DATA
fig = plt.figure(figsize=(9,10))

skew = SkewT(fig, rotation = 45) #rotate the temp axis by 45 degrees

# Plot adiabats, mixing ratio lines:
skew.plot_dry_adiabats(linewidth = 1)
skew.plot_moist_adiabats(linewidth = 1)
skew.plot_mixing_lines(linewidth = 1)

# Plot temperature data:
skew.plot(p, T, color = 'red', linewidth = 2)

# Plot dewpoint data:
skew.plot(p, Td, color = 'green', linewidth = 2)

# Plot wind data:
skew.plot_barbs(p[p_index], u[p_index], v[p_index], color = 'black')

######################################################
# Plot extra data:                                   

#Parcel profile:
skew.plot(p, parcel_profile, 'black', linewidth = 2)

#Shade areas of CAPE and CIN:
skew.shade_cin(p, T, parcel_profile)
skew.shade_cape(p, T, parcel_profile)
######################################################

# Set axis limits:
skew.ax.set_ylim(1025, 100)
skew.ax.set_xlim(-40, 60)

# Set axis labels:
plt.xlabel(r'Temperature ($\degree$C)')
plt.ylabel('Log Pressure (hPa)')

# Add a hodograph:
ax_hod = inset_axes(skew.ax, '30%', '30%', loc = 1)
h = Hodograph(ax_hod, component_range = 80)
h.add_grid(increment = 20)
h.plot_colormapped(u[p_index], v[p_index], wind_speed[p_index])

##################################################################
# Add a table with indices calculated previously:
ax2 = fig.add_subplot(212)
ax2.text(0.01, 0,
         'LCL: {0} hPa\nLFC: {1} hPa\nEL: {2} hPa'.format(
             LCL_pressure, LFC_pressure, EL_pressure))

ax2.text(0.20, 0,
         'CAPE: {0} J/kg\nMUCAPE: {1} J/kg\nCIN: {2} J/kg'.format(
             CAPE, MUCAPE, CIN))

ax2.set_axis_off()
##################################################################

plt.show()

***

And there we have a darn good looking skew-T for summertime convection! CAPE and CIN are shaded, the hodograph is present, wind barbs are nicely spaced and readable, and there's a concise table for severe weather indices.

More can be done with MetPy to make an even more advanced skew-T diagram, but for most cases, this is more than enough. It is encouraged that you dig deeper into the MetPy documentation to make your skew-T diagram as fancy as you wish.

MetPy can do more than skew-Ts too! MetPy can also plot gridded data (e.g. ASOS stations, satellite data, model output, etc), radar data, meteograms, and more. It is recommended you check out the following link for examples: https://unidata.github.io/MetPy/latest/examples/index.html

---

Another useful module for plotting thermodynamic data is SHARPPy, which may be a topic of a future Tools of the Trade. Documentation for this can be found at: https://github.com/sharppy/SHARPpy